# descarga de lista de todas las mesas nacionales e internacionales, y guardar en json's

en esta seccion se descarga los jsons que nos servira para descargar todas las actas nacionales y extranjeras

In [ ]:
# descargar datos de url y guardar el json
import requests
import json
import os

base_url = "https://computo.oep.org.bo/"


nacional_response = requests.get(base_url + "geografiaNacional.json")
internacional_response = requests.get(base_url + "geografiaExtranjero.json")

# descargar el archivo geografiaNacional.json
if nacional_response.status_code == 200:
    with open("geografiaNacional.json", "w", encoding="utf-8") as nacional_file:
        json.dump(nacional_response.json(), nacional_file, ensure_ascii=False, indent=2)
        print("Archivo geografiaNacional.json descargado y guardado correctamente.")
else:
    print(f"Error al descargar geografiaNacional.json: {nacional_response.status_code}")

# Guardar el archivo geografiaExtranjero.json
if internacional_response.status_code == 200:
    with open("geografiaExtranjera.json", "w", encoding="utf-8") as internacional_file:
        json.dump(internacional_response.json(), internacional_file, ensure_ascii=False, indent=2)
        print("Archivo geografiaExtranjero.json descargado y guardado correctamente.")
else:
    print(f"Error al descargar geografiaExtranjera.json: {internacional_response.status_code}")

descargamos la libreria tqdm para tener el proceso

In [ ]:
!pip install tqdm

aqui creamos la descarga de las actas nacionales en 8 hilos, 8 imagens de actas a la vez, depende de tu red el timepo que tardara

In [ ]:
import json
import os
import requests
import base64
import time
import csv
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Configuración
JSON_FILE = "geografiaNacional.json"
BASE_DIR = "ACTAS"
LOG_FILE = "descargas.csv"
URL = "https://computo.oep.org.bo/api/v1/resultados/mesa"
MAX_THREADS = 8         # hilos paralelos
MIN_DELAY = 0.3         # pausa mínima entre descargas
MAX_DELAY = 0.7         # pausa máxima entre descargas

# Cargar JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# Crear carpeta base
os.makedirs(BASE_DIR, exist_ok=True)

# Función para contar total de mesas
def contar_mesas(json_data):
    total = 0
    for pais in json_data:
        for dep in pais.get("d", []):
            for prov in dep.get("p", []):
                for mun in prov.get("m", []):
                    for loc in mun.get("l", []):
                        for rec in loc.get("r", []):
                            total += len(rec.get("t", []))
    return total

TOTAL_MESAS = contar_mesas(data)

# Función para descargar una mesa individual
def descargar_mesa(mesa_id, mesa_num, rec_path):
    mesa_file = os.path.join(rec_path, f"mesa_{mesa_num}_{mesa_id}.jpg")
    
    if os.path.exists(mesa_file):
        return mesa_num, mesa_id, mesa_file, "Ya descargado"
    
    try:
        resp = requests.post(URL, json={"codigoMesa": mesa_id}, timeout=15)
        if resp.status_code == 200:
            data_resp = resp.json()
            estado = "Sin ACTA"
            for adj in data_resp.get("adjunto", []):
                if adj["tipo"] == "ACTA" and adj["valor"] != "false":
                    img_data = base64.b64decode(adj["valor"])
                    with open(mesa_file, "wb") as f:
                        f.write(img_data)
                    del img_data
                    estado = "OK"
                    break
            return mesa_num, mesa_id, mesa_file, estado
        else:
            return mesa_num, mesa_id, "", f"Error HTTP {resp.status_code}"
    except Exception as e:
        return mesa_num, mesa_id, "", f"Error {e}"

# Preparar lista de tareas
tareas = []
for pais in data:
    pais_path = os.path.join(BASE_DIR, pais["n"])
    os.makedirs(pais_path, exist_ok=True)
    for dep in pais.get("d", []):
        dep_path = os.path.join(pais_path, dep["n"])
        os.makedirs(dep_path, exist_ok=True)
        for prov in dep.get("p", []):
            prov_path = os.path.join(dep_path, prov["n"])
            os.makedirs(prov_path, exist_ok=True)
            for mun in prov.get("m", []):
                mun_path = os.path.join(prov_path, mun["n"])
                os.makedirs(mun_path, exist_ok=True)
                for loc in mun.get("l", []):
                    loc_path = os.path.join(mun_path, loc["n"])
                    os.makedirs(loc_path, exist_ok=True)
                    for rec in loc.get("r", []):
                        rec_path = os.path.join(loc_path, rec["n"])
                        os.makedirs(rec_path, exist_ok=True)
                        for mesa in rec.get("t", []):
                            tareas.append((mesa["i"], mesa["n"], rec_path))

# Ejecutar descargas paralelas
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor, \
     open(LOG_FILE, "a", newline="", encoding="utf-8") as log:

    writer = csv.writer(log)
    if os.stat(LOG_FILE).st_size == 0:
        writer.writerow(["Mesa_Num", "Mesa_ID", "Ruta", "Estado"])

    futures = {executor.submit(descargar_mesa, tid, tnum, tpath): (tnum, tid) for tid, tnum, tpath in tareas}

    with tqdm(total=TOTAL_MESAS, desc="Descargando mesas") as pbar:
        for fut in as_completed(futures):
            mesa_num, mesa_id, mesa_file, estado = fut.result()
            writer.writerow([mesa_num, mesa_id, mesa_file, estado])
            # Pausa aleatoria pequeña para proteger servidor
            time.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
            pbar.update(1)

print("Descarga finalizada ✅")


actas internacionales, igual en 8 hilos concurrentes

In [ ]:
import json
import os
import requests
import base64
import time
import csv
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# Configuración
JSON_FILE = "geografiaExtranjera.json"
BASE_DIR = "ACTAS"
LOG_FILE = "descargas_extranjera.csv"
URL = "https://computo.oep.org.bo/api/v1/resultados/mesa"
MAX_THREADS = 8         # hilos paralelos
MIN_DELAY = 0.3         # pausa mínima entre descargas
MAX_DELAY = 0.7         # pausa máxima entre descargas

# Cargar JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# Crear carpeta base
os.makedirs(BASE_DIR, exist_ok=True)

# Función para contar total de mesas
def contar_mesas(json_data):
    total = 0
    for pais in json_data:
        for ciudad in pais.get("c", []):
            for recinto in ciudad.get("r", []):
                total += len(recinto.get("t", []))
    return total

TOTAL_MESAS = contar_mesas(data)

# Función para descargar una mesa individual
def descargar_mesa(mesa_id, mesa_num, rec_path):
    mesa_file = os.path.join(rec_path, f"mesa_{mesa_num}_{mesa_id}.jpg")
    
    if os.path.exists(mesa_file):
        return mesa_num, mesa_id, mesa_file, "Ya descargado"
    
    try:
        resp = requests.post(URL, json={"codigoMesa": mesa_id}, timeout=15)
        if resp.status_code == 200:
            data_resp = resp.json()
            estado = "Sin ACTA"
            for adj in data_resp.get("adjunto", []):
                if adj["tipo"] == "ACTA" and adj["valor"] != "false":
                    img_data = base64.b64decode(adj["valor"])
                    with open(mesa_file, "wb") as f:
                        f.write(img_data)
                    del img_data
                    estado = "OK"
                    break
            return mesa_num, mesa_id, mesa_file, estado
        else:
            return mesa_num, mesa_id, "", f"Error HTTP {resp.status_code}"
    except Exception as e:
        return mesa_num, mesa_id, "", f"Error {e}"

# Preparar lista de tareas
tareas = []
for pais in data:
    pais_path = os.path.join(BASE_DIR, pais["n"])
    os.makedirs(pais_path, exist_ok=True)
    for ciudad in pais.get("c", []):
        ciudad_path = os.path.join(pais_path, ciudad["n"])
        os.makedirs(ciudad_path, exist_ok=True)
        for recinto in ciudad.get("r", []):
            recinto_path = os.path.join(ciudad_path, recinto["n"])
            os.makedirs(recinto_path, exist_ok=True)
            for mesa in recinto.get("t", []):
                tareas.append((mesa["i"], mesa["n"], recinto_path))

# Ejecutar descargas paralelas
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor, \
     open(LOG_FILE, "a", newline="", encoding="utf-8") as log:

    writer = csv.writer(log)
    if os.stat(LOG_FILE).st_size == 0:
        writer.writerow(["Mesa_Num", "Mesa_ID", "Ruta", "Estado"])

    futures = {executor.submit(descargar_mesa, tid, tnum, tpath): (tnum, tid) for tid, tnum, tpath in tareas}

    with tqdm(total=TOTAL_MESAS, desc="Descargando mesas") as pbar:
        for fut in as_completed(futures):
            mesa_num, mesa_id, mesa_file, estado = fut.result()
            writer.writerow([mesa_num, mesa_id, mesa_file, estado])
            # Pausa aleatoria pequeña para proteger servidor
            time.sleep(random.uniform(MIN_DELAY, MAX_DELAY))
            pbar.update(1)

print("Descarga finalizada ✅")
